In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

# ======================================================================================
# konfiguracia hry
# ======================================================================================
HISTORY_LENGTH = 2           # Number of past moves to consider
GENOME_LENGTH = 2 ** (2 * HISTORY_LENGTH)  # 16 for HISTORY_LENGTH=2
NUM_ROUNDS = 100            # Number of rounds per IPD game

# ======================================================================================
# parametre genetickeho algoritmu
# ======================================================================================
POPULATION_SIZE = 300       # pocet individualov v populacii
MUTATION_RATE = 0.2         # sanca mutacie
CROSSOVER_RATE = 0.8        # sanca krizenia
TOURNAMENT_SIZE = 4         # pocet jedincov v turnaji
ELITISM_SIZE = 5            # pocet elitnych jedincov
MAX_GENERATIONS = 150       # pocet generacii

# ======================================================================================
# tabulka iterovaneho vaznovho dilema
# ======================================================================================
PAYOFF = {
    (0, 0): (3, 3),  # obaja spolupracuju
    (0, 1): (0, 5),  # agent spolupracuje, oponent zradi
    (1, 0): (5, 0),  # agent zradi, oponent spolupracuje
    (1, 1): (1, 1)   # obaja zradia
}

# ======================================================================================
# strategie superov
# ======================================================================================
def always_cooperate(_, __):
    # vzdy spolupracovat
    return 0

def always_defect(_, __):
    # vzdy zradit
    return 1

def tit_for_tat(_, opp_hist):
    # spolupracovat na zaciatku potom kopirovat
    return opp_hist[-1] if opp_hist else 0

def random_strategy(_, __):
    # nahoda
    return random.randint(0, 1)

OPPONENTS = [
    ("AlwaysCooperate", always_cooperate),
    ("AlwaysDefect", always_defect),
    ("TitForTat", tit_for_tat),
    ("Random", random_strategy)
]

# ======================================================================================
# inicializacia deap toolboxu
# ======================================================================================
toolbox = base.Toolbox()

# registracia funkcii pre generovanie genov, jedincov a populacie
toolbox.register("gene_maker", random.randint, MIN_HEIGHT, MAX_HEIGHT)
toolbox.register("individual_creator", tools.initRepeat, creator.Individual, 
                 toolbox.gene_maker, n=GENOME_LENGTH)
toolbox.register("population", tools.initRepeat, list, toolbox.individual_creator)

# registracia operatorov genetickeho algoritmu
toolbox.register("evaluate", sophisticated_fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=MIN_HEIGHT, up=MAX_HEIGHT, 
                 indpb=CROSSOVER_RATE)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)
toolbox.register("elitism", tools.selBest, k=ELITISM_SIZE)

# inicializacia sledovania statistik
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)  # sledovanie priemereho fitness
stats.register("min", np.min)   # sledovanie minimaly fitness
stats.register("max", np.max)   # sledovanie maximaly fitness


# ======================================================================================
#
# ======================================================================================

if __name__ == "__main__":
    print(f"asdf")
    # spustenie genetickeho algoritmu
    algorithms.eaSimple(population, 
                        toolbox,
                        cxpb=CROSSOVER_RATE,
                        mutpb=MUTATION_RATE,
                        ngen=MAX_GENERATIONS,
                        stats=stats,
                        halloffame=hof,
                        verbose=True)
    
    

asdf
